In [1]:
import pandas as pd
import numpy as np

In [2]:
import os

directory = '/Users/pinakshome/Downloads/store-sales-time-series-forecasting/'

for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        file_path = os.path.join(directory, filename)
        df_name = os.path.splitext(filename)[0]  # Use the filename (without extension) as the DataFrame name
        globals()[df_name] = pd.read_csv(file_path)

<h1> <B> PreProcessing From Kaggle LightGBM notebook Reference <B> </h1>

In [5]:
### Unique Values
train.nunique()

id             3000888
date              1684
store_nbr           54
family              33
sales           379610
onpromotion        362
dtype: int64

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 137.4+ MB


In [9]:
############# No Duplicates
duplicates = df_train[df_train.duplicated()]

print("Duplicate rows:")
print(duplicates)

Duplicate rows:
Empty DataFrame
Columns: [id, date, store_nbr, family, sales, onpromotion]
Index: []


In [156]:
# Check for duplicates based on column combination
duplicates = df_train1[df_train1.duplicated(['date','family','store_nbr','sales','onpromotion'])]

print("Duplicate rows based on 'Column1' and 'Column2':")
print(duplicates)


Duplicate rows based on 'Column1' and 'Column2':
Empty DataFrame
Columns: [id, date, store_nbr, family, sales, onpromotion]
Index: []


In [10]:
df_train.head(-100)

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.000000,0
1,1,2013-01-01,1,BABY CARE,0.000000,0
2,2,2013-01-01,1,BEAUTY,0.000000,0
3,3,2013-01-01,1,BEVERAGES,0.000000,0
4,4,2013-01-01,1,BOOKS,0.000000,0
...,...,...,...,...,...,...
3000783,3000783,2017-08-15,6,PLAYERS AND ELECTRONICS,15.000000,0
3000784,3000784,2017-08-15,6,POULTRY,434.570000,0
3000785,3000785,2017-08-15,6,PREPARED FOODS,102.341995,0
3000786,3000786,2017-08-15,6,PRODUCE,2073.636000,5


In [ ]:
print(df_train.isnull().sum())

In [26]:
df_train.describe()

,id,store_nbr,sales,onpromotion
count,3.000888e+06,3.000888e+06,3.000888e+06,3.000888e+06
mean,1.500444e+06,2.750000e+01,3.577757e+02,2.602770e+00
std,8.662819e+05,1.558579e+01,1.101998e+03,1.221888e+01
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,7.502218e+05,1.400000e+01,0.000000e+00,0.000000e+00
50%,1.500444e+06,2.750000e+01,1.100000e+01,0.000000e+00
75%,2.250665e+06,4.100000e+01,1.958473e+02,0.000000e+00
max,3.000887e+06,5.400000e+01,1.247170e+05,7.410000e+02


In [33]:
###Checking non-null values
df_train.count()

id             3000888
date           3000888
store_nbr      3000888
family         3000888
sales          3000888
onpromotion    3000888
dtype: int64

In [112]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


# Select features and target for training
y = df_train['sales']
df_train = df_train.drop(['sales'],axis = 1)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(df_train, y, test_size=0.2, random_state=42)

# Initialize and train the Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the validation set
y_val_pred = model.predict(X_val)

# Calculate the Mean Squared Error (MSE) for the validation set predictions
mse = mean_squared_error(y_val, y_val_pred)

mse

523864.739374515

In [3]:
train['date'] = pd.to_datetime(train['date'])

In [4]:
test['date'] = pd.to_datetime(test['date'])

In [5]:
num_family = train.family.nunique()
num_store = train.store_nbr.nunique()
num_ts = train.groupby(["store_nbr", "family"]).ngroups
train_start = train.date.min().date()
train_end = train.date.max().date()
num_train_date = train.date.nunique()
train_len = (train_end - train_start).days + 1
test_start = test.date.min().date()
test_end = test.date.max().date()
num_test_date = test.date.nunique()
test_len = (test_end - test_start).days + 1

In [6]:
# Print all variables
print(f"Number of unique families: {num_family}")
print(f"Number of unique stores: {num_store}")
print(f"Number of unique time series (store-family combinations): {num_ts}")
print(f"Training start date: {train_start}")
print(f"Training end date: {train_end}")
print(f"Number of unique training dates: {num_train_date}")
print(f"Length of training period (in days): {train_len}")
print(f"Test start date: {test_start}")
print(f"Test end date: {test_end}")
print(f"Number of unique test dates: {num_test_date}")
print(f"Length of test period (in days): {test_len}")

Number of unique families: 33
Number of unique stores: 54
Number of unique time series (store-family combinations): 1782
Training start date: 2013-01-01
Training end date: 2017-08-15
Number of unique training dates: 1684
Length of training period (in days): 1688
Test start date: 2017-08-16
Test end date: 2017-08-31
Number of unique test dates: 16
Length of test period (in days): 16


In [7]:
# Check unique store-family-date combinations
unique_combinations = train.drop_duplicates(subset=["store_nbr", "family", "date"])

# Number of unique combinations
num_unique_combinations = unique_combinations.shape[0]

# Print the number of unique combinations
print(f"Number of unique store-family-date combinations: {num_unique_combinations}")

Number of unique store-family-date combinations: 3000888


In [8]:
missing_dates = pd.date_range(train_start, train_end).difference(train.date.unique())
missing_dates = missing_dates.strftime("%Y-%m-%d").tolist()

unique_dp_count = train.groupby(["store_nbr", "family"]).date.count().unique().tolist()

In [9]:
print(unique_dp_count)

[1684]


In [10]:
print(missing_dates)

['2013-12-25', '2014-12-25', '2015-12-25', '2016-12-25']


In [11]:
####### REINDEXING and LINEAR EXTRAPOLATING ON ID
# reindex training data
multi_idx = pd.MultiIndex.from_product(
    [pd.date_range(train_start, train_end), train.store_nbr.unique(), train.family.unique()],
    names=["date", "store_nbr", "family"],
)
train = train.set_index(["date", "store_nbr", "family"]).reindex(multi_idx).reset_index()

# fill missing values with 0s
train[["sales", "onpromotion"]] = train[["sales", "onpromotion"]].fillna(0.)
train.id = train.id.interpolate(method="linear") # interpolate linearly as a filler for the 'id'

In [12]:
###### OIL DATES MISSING CHECK
missing_oil_dates = pd.date_range(train_start, test_end).difference(oil.date)
num_missing_oil_dates = len(missing_oil_dates)
num_wknd_missing = (missing_oil_dates.weekday >= 5).sum()
total_num_wknd = (pd.date_range(train_start, test_end).weekday >= 5).sum()

In [13]:
print(missing_oil_dates)
print(num_missing_oil_dates)
print(num_wknd_missing)
print(total_num_wknd)

DatetimeIndex(['2013-01-05', '2013-01-06', '2013-01-12', '2013-01-13',
               '2013-01-19', '2013-01-20', '2013-01-26', '2013-01-27',
               '2013-02-02', '2013-02-03',
               ...
               '2017-07-29', '2017-07-30', '2017-08-05', '2017-08-06',
               '2017-08-12', '2017-08-13', '2017-08-19', '2017-08-20',
               '2017-08-26', '2017-08-27'],
              dtype='datetime64[ns]', length=486, freq=None)
486
486
486


In [16]:
oil['date'] = pd.to_datetime(oil['date'])

In [18]:
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-05,NaN


In [19]:
# reindex oil data
oil = oil.merge(
    pd.DataFrame({"date": pd.date_range(train_start, test_end)}),
    on="date",
    how="outer",
).sort_values("date", ignore_index=True)

# fill missing values using linear interpolation
oil.dcoilwtico = oil.dcoilwtico.interpolate(method="linear", limit_direction="both")

In [21]:
oil.head()

,date,dcoilwtico
0,2013-01-01,93.140000
1,2013-01-02,93.140000
2,2013-01-03,92.970000
3,2013-01-04,93.120000
4,2013-01-05,93.146667


In [22]:
###### OIL DATES MISSING CHECK
missing_oil_dates = pd.date_range(train_start, test_end).difference(oil.date)
num_missing_oil_dates = len(missing_oil_dates)
num_wknd_missing = (missing_oil_dates.weekday >= 5).sum()
total_num_wknd = (pd.date_range(train_start, test_end).weekday >= 5).sum()

In [23]:
print(missing_oil_dates)
print(num_missing_oil_dates)
print(num_wknd_missing)
print(total_num_wknd)

DatetimeIndex([], dtype='datetime64[ns]', freq='D')
0
0
486


In [24]:
##### Oil df fixed

In [26]:
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [34]:
transactions['date'] = pd.to_datetime(transactions['date'])

In [27]:
num_zero_sales = (train.groupby(["date", "store_nbr"]).sales.sum().eq(0)).sum()
total_rec = num_store * train_len
curr_rec = len(transactions.index)
missing_rec = total_rec - curr_rec - num_zero_sales

In [28]:
print(num_zero_sales)

7546


In [29]:
print(total_rec)

91152


In [32]:
print(missing_rec)

118


In [35]:
# compute total sales for each store
store_sales = train.groupby(["date", "store_nbr"]).sales.sum().reset_index()

# reindex transaction data
transactions = transactions.merge(
    store_sales,
    on=["date", "store_nbr"],
    how="outer",
).sort_values(["date", "store_nbr"], ignore_index=True)

# fill missing values with 0s for days with zero sales
transactions.loc[transactions.sales.eq(0), "transactions"] = 0.
transactions = transactions.drop(columns=["sales"])

# fill remaining missing values using linear interpolation
transactions.transactions = transactions.groupby("store_nbr", group_keys=False).transactions.apply(
    lambda x: x.interpolate(method="linear", limit_direction="both")
)

In [37]:
national_locale_name = sorted(holidays_events[holidays_events.locale.eq("National")].locale_name.unique().tolist())
regional_locale_name = sorted(holidays_events[holidays_events.locale.eq("Regional")].locale_name.unique().tolist())
local_locale_name = sorted(holidays_events[holidays_events.locale.eq("Local")].locale_name.unique().tolist())

In [38]:
print(national_locale_name)

['Ecuador']


In [39]:
print(regional_locale_name)

['Cotopaxi', 'Imbabura', 'Santa Elena', 'Santo Domingo de los Tsachilas']


In [40]:
print(local_locale_name)

['Ambato', 'Cayambe', 'Cuenca', 'El Carmen', 'Esmeraldas', 'Guaranda', 'Guayaquil', 'Ibarra', 'Latacunga', 'Libertad', 'Loja', 'Machala', 'Manta', 'Puyo', 'Quevedo', 'Quito', 'Riobamba', 'Salinas', 'Santo Domingo']
